# Classification with deep learning

The classification solution tries to address the problem using deep learning. I just tried it out of curiosity:

*Train a learning model that assigns each expense transaction to one of the set of predefined categories and evaluate it against the validation data provided. The set of categories are those found in the "category" column in the training data. Report on accuracy and at least one other performance metric.*

For this, I tried the following:
- **Using a simple 3-layer ANN with all features (non-text and text, with text vectorized using scikit learn's TfIdfVectorizer)**
- **Using a simple 3-layer ANN with only text features (with text vectorized using Keras's text to matrix binary vectorizer)**
- **Using a RNN with LSTM with a pre-trained embedding layer and only text features (with text vectorized using Keras's text to matrix binary vectorizer)**


## Necessary imports

In [65]:
import pandas as pd
import numpy as np
from collections import Counter
import itertools

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from keras import models, layers
from keras.utils import np_utils
from keras import preprocessing as kreprocessing
import os

Read data nd preprocess

In [109]:
df_train = pd.read_csv('training_data_example.csv')
df_val = pd.read_csv('validation_data_example.csv')
df_employee = pd.read_csv('employee.csv')

In [110]:
def pre_process(df, columns_to_drop=['date',
                                    'category', 
                                    'tax amount',                                 
                                    'expense description']):
    
    df['day_of_week'] = pd.to_datetime(df['date']).apply(lambda x: x.weekday()).astype(str) # str so that treated as categoical
    df['month'] = pd.to_datetime(df['date']).apply(lambda x: x.month).astype(str)
    df = pd.merge(df, df_employee[['employee id', 'role']], how='inner', on=['employee id'])
    df['employee id'] = df['employee id'].astype(str)
    df = df.drop(columns_to_drop, axis=1)
    
    # one-hot encode the categorical variables
    df = pd.get_dummies(df)
    
    df['pre-tax amount'] = preprocessing.minmax_scale(df[['pre-tax amount']])
    
    return df

Ready the pre-processed data

In [111]:
x_train = pre_process(df_train)
x_val = pre_process(df_val)
x_train, x_val = x_train.align(x_val, join='left', axis=1)
x_val = x_val.fillna(0)

Process text features separately

In [112]:
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(df_train['expense description'])
x_train_tfidf = vectorizer.transform(df_train['expense description']).toarray()
x_val_tfidf = vectorizer.transform(df_val['expense description']).toarray()
# x_train_tfidf.shape

Concatenate text and non-text features

In [113]:
x_train = np.concatenate((x_train.values, x_train_tfidf), axis=1)
x_val = np.concatenate((x_val.values, x_val_tfidf), axis=1)

Generate vectorized output labels for feeding to Keras.

In [114]:
lencoder = LabelEncoder()
lencoder.fit(df_train['category'])
names = set(df_val['category']) # label names to be used later
y_train = lencoder.transform(df_train['category'])
y_val = lencoder.transform(df_val['category'])
dummy_y_train = np_utils.to_categorical(y_train)
dummy_y_val = np_utils.to_categorical(y_val)

## Approach 1

### Using a simple 3-layer ANN with all features (non-text and text, with text vectorized using scikit learn's TfIdfVectorizer)

In [115]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 16)                832       
_________________________________________________________________
dense_42 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_43 (Dense)             (None, 5)                 85        
Total params: 1,189
Trainable params: 1,189
Non-trainable params: 0
_________________________________________________________________


In [43]:
epochs = 200
history = model.fit(x_train,
                    dummy_y_train,
                    epochs=epochs,
                    batch_size=12,
                    validation_data=(x_val, dummy_y_val))

Train on 24 samples, validate on 12 samples
Epoch 1/200
24/24 [==============================] - 0s 14ms/step - loss: 1.6079 - acc: 0.1250 - val_loss: 1.6118 - val_acc: 0.2500
Epoch 2/200
24/24 [==============================] - 0s 324us/step - loss: 1.5794 - acc: 0.1667 - val_loss: 1.6026 - val_acc: 0.1667
Epoch 3/200
24/24 [==============================] - 0s 628us/step - loss: 1.5615 - acc: 0.2500 - val_loss: 1.5947 - val_acc: 0.1667
Epoch 4/200
24/24 [==============================] - 0s 308us/step - loss: 1.5471 - acc: 0.2500 - val_loss: 1.5873 - val_acc: 0.0833
Epoch 5/200
24/24 [==============================] - 0s 332us/step - loss: 1.5331 - acc: 0.3333 - val_loss: 1.5800 - val_acc: 0.1667
Epoch 6/200
24/24 [==============================] - 0s 526us/step - loss: 1.5206 - acc: 0.4167 - val_loss: 1.5730 - val_acc: 0.3333
Epoch 7/200
24/24 [==============================] - 0s 418us/step - loss: 1.5087 - acc: 0.5000 - val_loss: 1.5665 - val_acc: 0.3333
Epoch 8/200
24/24 [=======

Epoch 62/200
24/24 [==============================] - 0s 343us/step - loss: 0.8262 - acc: 0.7917 - val_loss: 1.2089 - val_acc: 0.7500
Epoch 63/200
24/24 [==============================] - 0s 370us/step - loss: 0.8135 - acc: 0.7917 - val_loss: 1.2033 - val_acc: 0.7500
Epoch 64/200
24/24 [==============================] - 0s 388us/step - loss: 0.8055 - acc: 0.7917 - val_loss: 1.1976 - val_acc: 0.7500
Epoch 65/200
24/24 [==============================] - 0s 627us/step - loss: 0.7914 - acc: 0.7917 - val_loss: 1.1922 - val_acc: 0.7500
Epoch 66/200
24/24 [==============================] - 0s 387us/step - loss: 0.7783 - acc: 0.7917 - val_loss: 1.1860 - val_acc: 0.7500
Epoch 67/200
24/24 [==============================] - 0s 574us/step - loss: 0.7668 - acc: 0.7917 - val_loss: 1.1800 - val_acc: 0.7500
Epoch 68/200
24/24 [==============================] - 0s 320us/step - loss: 0.7555 - acc: 0.7917 - val_loss: 1.1744 - val_acc: 0.7500
Epoch 69/200
24/24 [==============================] - 0s 361us

Epoch 123/200
24/24 [==============================] - 0s 421us/step - loss: 0.2979 - acc: 0.9167 - val_loss: 0.8615 - val_acc: 0.9167
Epoch 124/200
24/24 [==============================] - 0s 385us/step - loss: 0.2916 - acc: 0.9167 - val_loss: 0.8556 - val_acc: 0.9167
Epoch 125/200
24/24 [==============================] - 0s 338us/step - loss: 0.2864 - acc: 0.9167 - val_loss: 0.8518 - val_acc: 0.9167
Epoch 126/200
24/24 [==============================] - 0s 312us/step - loss: 0.2819 - acc: 0.9167 - val_loss: 0.8474 - val_acc: 0.9167
Epoch 127/200
24/24 [==============================] - 0s 409us/step - loss: 0.2782 - acc: 0.9167 - val_loss: 0.8433 - val_acc: 0.9167
Epoch 128/200
24/24 [==============================] - 0s 501us/step - loss: 0.2715 - acc: 0.9167 - val_loss: 0.8398 - val_acc: 0.9167
Epoch 129/200
24/24 [==============================] - 0s 447us/step - loss: 0.2680 - acc: 0.9167 - val_loss: 0.8374 - val_acc: 0.9167
Epoch 130/200
24/24 [==============================] - 

Epoch 184/200
24/24 [==============================] - 0s 431us/step - loss: 0.0942 - acc: 1.0000 - val_loss: 0.7043 - val_acc: 0.9167
Epoch 185/200
24/24 [==============================] - 0s 563us/step - loss: 0.0917 - acc: 1.0000 - val_loss: 0.7039 - val_acc: 0.9167
Epoch 186/200
24/24 [==============================] - 0s 408us/step - loss: 0.0907 - acc: 1.0000 - val_loss: 0.7024 - val_acc: 0.9167
Epoch 187/200
24/24 [==============================] - 0s 333us/step - loss: 0.0874 - acc: 1.0000 - val_loss: 0.7034 - val_acc: 0.9167
Epoch 188/200
24/24 [==============================] - 0s 313us/step - loss: 0.0854 - acc: 1.0000 - val_loss: 0.7043 - val_acc: 0.9167
Epoch 189/200
24/24 [==============================] - 0s 290us/step - loss: 0.0831 - acc: 1.0000 - val_loss: 0.7029 - val_acc: 0.9167
Epoch 190/200
24/24 [==============================] - 0s 386us/step - loss: 0.0809 - acc: 1.0000 - val_loss: 0.7020 - val_acc: 0.9167
Epoch 191/200
24/24 [==============================] - 

### From the logs above, the training accuracy is 100% and the validation accuracy is 91.67%

In [116]:
np.argmax(model.predict(x_val), axis=1)

array([3, 4, 4, 0, 0, 3, 4, 3, 4, 3, 1, 3])

In [117]:
y_val

array([4, 2, 0, 3, 3, 4, 2, 2, 2, 2, 2, 2])

## Approach 2

### Using a simple 3-layer ANN with only text features (with text vectorized using Keras's text to matrix binary vectorizer)

In [118]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_train['expense description'])

In [119]:
x_train_tfidf_keras = tokenizer.texts_to_matrix(df_train['expense description'], mode='binary')
x_val_tfidf_keras = tokenizer.texts_to_matrix(df_val['expense description'], mode='binary')
x_train_tfidf_keras.shape

(24, 10000)

In [120]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 16)                160016    
_________________________________________________________________
dense_45 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_46 (Dense)             (None, 5)                 85        
Total params: 160,373
Trainable params: 160,373
Non-trainable params: 0
_________________________________________________________________


In [59]:
epochs = 200
history = model.fit(x_train_tfidf_keras,
                    dummy_y_train,
                    epochs=epochs,
                    batch_size=12,
                    validation_data=(x_val_tfidf_keras, dummy_y_val))

Train on 24 samples, validate on 12 samples
Epoch 1/200
24/24 [==============================] - 0s 19ms/step - loss: 1.6093 - acc: 0.3750 - val_loss: 1.5909 - val_acc: 0.5000
Epoch 2/200
24/24 [==============================] - 0s 547us/step - loss: 1.5923 - acc: 0.4167 - val_loss: 1.5764 - val_acc: 0.6667
Epoch 3/200
24/24 [==============================] - 0s 772us/step - loss: 1.5797 - acc: 0.5833 - val_loss: 1.5647 - val_acc: 0.6667
Epoch 4/200
24/24 [==============================] - 0s 729us/step - loss: 1.5701 - acc: 0.6250 - val_loss: 1.5547 - val_acc: 0.6667
Epoch 5/200
24/24 [==============================] - 0s 842us/step - loss: 1.5604 - acc: 0.5833 - val_loss: 1.5448 - val_acc: 0.6667
Epoch 6/200
24/24 [==============================] - 0s 841us/step - loss: 1.5512 - acc: 0.6667 - val_loss: 1.5351 - val_acc: 0.6667
Epoch 7/200
24/24 [==============================] - 0s 836us/step - loss: 1.5421 - acc: 0.6667 - val_loss: 1.5246 - val_acc: 0.6667
Epoch 8/200
24/24 [=======

Epoch 62/200
24/24 [==============================] - 0s 3ms/step - loss: 0.7207 - acc: 0.9583 - val_loss: 0.7289 - val_acc: 0.8333
Epoch 63/200
24/24 [==============================] - 0s 746us/step - loss: 0.7050 - acc: 0.9583 - val_loss: 0.7160 - val_acc: 0.8333
Epoch 64/200
24/24 [==============================] - 0s 772us/step - loss: 0.6892 - acc: 0.9583 - val_loss: 0.7032 - val_acc: 0.8333
Epoch 65/200
24/24 [==============================] - 0s 1ms/step - loss: 0.6745 - acc: 0.9583 - val_loss: 0.6910 - val_acc: 0.8333
Epoch 66/200
24/24 [==============================] - 0s 849us/step - loss: 0.6614 - acc: 0.9583 - val_loss: 0.6800 - val_acc: 0.9167
Epoch 67/200
24/24 [==============================] - 0s 812us/step - loss: 0.6449 - acc: 0.9583 - val_loss: 0.6678 - val_acc: 0.9167
Epoch 68/200
24/24 [==============================] - 0s 812us/step - loss: 0.6304 - acc: 1.0000 - val_loss: 0.6572 - val_acc: 0.9167
Epoch 69/200
24/24 [==============================] - 0s 925us/ste

24/24 [==============================] - 0s 666us/step - loss: 0.1366 - acc: 1.0000 - val_loss: 0.2996 - val_acc: 0.9167
Epoch 124/200
24/24 [==============================] - 0s 721us/step - loss: 0.1322 - acc: 1.0000 - val_loss: 0.2955 - val_acc: 0.9167
Epoch 125/200
24/24 [==============================] - 0s 609us/step - loss: 0.1282 - acc: 1.0000 - val_loss: 0.2919 - val_acc: 0.9167
Epoch 126/200
24/24 [==============================] - 0s 639us/step - loss: 0.1243 - acc: 1.0000 - val_loss: 0.2895 - val_acc: 0.9167
Epoch 127/200
24/24 [==============================] - 0s 918us/step - loss: 0.1208 - acc: 1.0000 - val_loss: 0.2859 - val_acc: 0.9167
Epoch 128/200
24/24 [==============================] - 0s 1ms/step - loss: 0.1169 - acc: 1.0000 - val_loss: 0.2835 - val_acc: 0.9167
Epoch 129/200
24/24 [==============================] - 0s 1ms/step - loss: 0.1134 - acc: 1.0000 - val_loss: 0.2800 - val_acc: 0.9167
Epoch 130/200
24/24 [==============================] - 0s 990us/step - lo

Epoch 184/200
24/24 [==============================] - 0s 585us/step - loss: 0.0207 - acc: 1.0000 - val_loss: 0.1535 - val_acc: 0.9167
Epoch 185/200
24/24 [==============================] - 0s 725us/step - loss: 0.0199 - acc: 1.0000 - val_loss: 0.1521 - val_acc: 0.9167
Epoch 186/200
24/24 [==============================] - 0s 692us/step - loss: 0.0192 - acc: 1.0000 - val_loss: 0.1486 - val_acc: 0.9167
Epoch 187/200
24/24 [==============================] - 0s 529us/step - loss: 0.0187 - acc: 1.0000 - val_loss: 0.1475 - val_acc: 0.9167
Epoch 188/200
24/24 [==============================] - 0s 608us/step - loss: 0.0179 - acc: 1.0000 - val_loss: 0.1466 - val_acc: 0.9167
Epoch 189/200
24/24 [==============================] - 0s 618us/step - loss: 0.0173 - acc: 1.0000 - val_loss: 0.1443 - val_acc: 0.9167
Epoch 190/200
24/24 [==============================] - 0s 758us/step - loss: 0.0167 - acc: 1.0000 - val_loss: 0.1434 - val_acc: 0.9167
Epoch 191/200
24/24 [==============================] - 

### From the logs above, the training accuracy is 100% and the validation accuracy is 91.67%

## Approach 3

### Using a RNN with LSTM with a pre-trained embedding layer (GLOVE) and only text features (with text vectorized using Keras's text to matrix binary vectorizer)

In [121]:
glove_dir = 'glove.6B'
embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [122]:
max_len=500
embedding_dimension=100

word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(df_train['expense description'])
val_sequences = tokenizer.texts_to_sequences(df_val['expense description'])

train_data = kreprocessing.sequence.pad_sequences(train_sequences, maxlen=max_len)
val_data = kreprocessing.sequence.pad_sequences(val_sequences, maxlen=max_len)

In [99]:
train_data.shape

(24, 500)

In [100]:
max_words = 10000
embedding_matrix = np.zeros((max_words, embedding_dimension))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [101]:
model = models.Sequential()
model.add(layers.Embedding(max_words, embedding_dimension, input_length=max_len))
model.add(layers.LSTM(32))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_40 (Dense)             (None, 5)                 165       
Total params: 1,017,189
Trainable params: 1,017,189
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [103]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [104]:
history = model.fit(train_data, dummy_y_train,
                    epochs=20,
                    batch_size=6,
                    validation_data=(val_data, dummy_y_val))

Train on 24 samples, validate on 12 samples
Epoch 1/20
24/24 [==============================] - 3s 123ms/step - loss: 1.5062 - acc: 0.3750 - val_loss: 1.3186 - val_acc: 0.7500
Epoch 2/20
24/24 [==============================] - 2s 71ms/step - loss: 1.2632 - acc: 0.7917 - val_loss: 1.1607 - val_acc: 0.8333
Epoch 3/20
24/24 [==============================] - 2s 71ms/step - loss: 1.0976 - acc: 0.9167 - val_loss: 1.0268 - val_acc: 0.8333
Epoch 4/20
24/24 [==============================] - 2s 70ms/step - loss: 0.9557 - acc: 0.9167 - val_loss: 0.9094 - val_acc: 0.8333
Epoch 5/20
24/24 [==============================] - 2s 70ms/step - loss: 0.8403 - acc: 0.9167 - val_loss: 0.8069 - val_acc: 0.8333
Epoch 6/20
24/24 [==============================] - 2s 72ms/step - loss: 0.7366 - acc: 0.9167 - val_loss: 0.7196 - val_acc: 0.8333
Epoch 7/20
24/24 [==============================] - 2s 73ms/step - loss: 0.6395 - acc: 0.9167 - val_loss: 0.6425 - val_acc: 0.8333
Epoch 8/20
24/24 [====================

### From the logs above, the training accuracy is 100% and the validation accuracy is 91.67%